# Feature Enginering 

The previous sections outline the fundamental ideas of machine learning, but all fo the exmaples assume that you have numerical data in a tidy, `[n_samples, n_features]` format. In the real world, data rarely comes in such a form. With this in mind, one of the more important steps in using machine learning in practice is *feature engineering*: that is, taking whatever information you have about your problem and turning it into numbers that you can use to build your feature matrix.

In this section, we will cover a few common examples of feature engineering tasks:

features  for representing *categorical data*, features for representing *text*, and *features* for representing *images*. Additionally, we will discuss *derived features* for increasing model complexity and *imputation* of missin data. Often this process is known as *vectorization*, as it involves converting arbitratry data into well-behaved vectors.

## Categorical Features 

One common type of non-numerical data is *categorical* data. For example, imagine you are exploring some data on housing prices, and along with numerical features like "price" and "rooms", you also have "neighborhood" information. For example, your data might look something like this:

In [1]:
data = [
    {'price': 850000, 'rooms': 4, 'neighborhood': 'Queen Anne'},
    {'price': 700000, 'rooms': 3, 'neighborhood': 'Fremont'},
    {'price': 650000, 'rooms': 3, 'neighborhood': 'Wallingford'},
    {'price': 600000, 'rooms': 2, 'neighborhood': 'Fremont'}
]

You might be tempted to encode this data with a straightforward numerical mapping:

In [2]:
{'Queen Anne': 1, 'Fremont': 2, 'Wallingford': 3}

{'Queen Anne': 1, 'Fremont': 2, 'Wallingford': 3}

It turns out that his is not generally a useful approach in Scikit-Learn: the package's models make the fundamental assumption that numerical features reflect algebraic quantities. Thus such a mapping would imply, for example, that *Queen Anne < Fremont < Wallingford* or even that *Wallingford - Queen Anne* = *Fremont*, which (niche demographic jokes aside) does not make such sense.

In this case, one proven technique is to use *one-hot encoding*, which effectively creates extra columns indicating the presence or absence of a category with a value or 1 or 0, respectively. When your data comes as list of dictionaries, Scikit-Learn's `DictVectorizer` will do this for you:

In [3]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False, dtype=int)
vec.fit_transform(data)

array([[     0,      1,      0, 850000,      4],
       [     1,      0,      0, 700000,      3],
       [     0,      0,      1, 650000,      3],
       [     1,      0,      0, 600000,      2]])

Notice that the 'neighborhood' column has been expanded into three separate columns, representing the three neighborhod labels, and that each row has a 1 in the column associated with its neighborhood. With these categorical features thus encodes, you can proceed as normal with fitting a Scikit-Learn model.

To see the meaning of each column, you can inspect the feature names:

In [4]:
vec.get_feature_names()

['neighborhood=Fremont',
 'neighborhood=Queen Anne',
 'neighborhood=Wallingford',
 'price',
 'rooms']

There is one clear disadvantage of this approach: if your category has many possible values, this can *greatly* increase the size of your dataset. However, because the encodes data contains mostly zeros, a sparse output can be a very efficient solution:

In [6]:
vec = DictVectorizer(sparse=True, dtype=int)
vec.fit_transform(data)

<4x5 sparse matrix of type '<class 'numpy.int32'>'
	with 12 stored elements in Compressed Sparse Row format>

Many (though not yer all) of the Scikit-Learn estimators accept such sparse inputs when fitting and evaluating models. `sklearn.preprocessing.OneHotEncoder` and `sklearn.feature-extraction.FeatureHasher` are two additional tools taht Scikiti-Learn includes to support this type of encoding.

## Text Features

Another common need in feature engineering is to convert text to a set of representative numerical values. For example, most automatic mining of social media data relies on some form of encoding the text as numbers. One of the simplest methods of encoding data is by *woed counts*: you take each snippet of text, count the ocurrences of each word within it, and put the results in a table.

For example, consider the foloowing set of three phrases:

In [7]:
sample = ['problem of evil', 'evil queen', 'horizon problem']

For a vectorization of this dta based on word count, we could construct a column representing the word "problem", the word "evil," the word "horizon", and so on. While doing this by hand would be possible, the tedium can be avoided byusing Scikit-Learn's `CountVectorizer`:

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(sample)
X

<3x5 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [10]:
import pandas as pd 

pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

,evil,horizon,of,problem,queen
0,1,0,1,1,0
1,1,0,0,0,1
2,0,1,0,1,0


There are some issues with this approach, however: the raw word counts lead to features which put too much weight on words that appear very frequently, and this ca be sub-optimal in some classification algorithms. One approach to fix this is known as *term frequency-inverse document frequency (TF-IDF)* which weights the word counts by a measure of how often they appear in the documents. The syntax for computing these features is similar to the previous example:

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer 
vec = TfidfVectorizer()
X = vec.fit_transform(sample)
pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

,evil,horizon,of,problem,queen
0,0.517856,0.000000,0.680919,0.517856,0.000000
1,0.605349,0.000000,0.000000,0.000000,0.795961
2,0.000000,0.795961,0.000000,0.605349,0.000000


## Image Features 

Another common need is to suitable encode *images* for machine learning analysis. The simplest approach is what we used for the digits data in the introduction, simply using the pixel values themselves. But depending on the application, such approaches may not be optimal.

A comprehensive sumamry of these features extraction techniques for images is well beyonc the scope of this section.